In [2]:
import pandas as pd
import sys
sys.path.append("../src")
from preprocessing import clean_text
from classifier import GPTJobDescriptionCategorizer
import os

In [3]:
df = pd.read_csv("../Engineer_20230826.csv")
df['JobDescription'] = df['JobDescription'].apply(clean_text)

In [7]:
classifier = GPTJobDescriptionCategorizer(
    job_postings_data=df, 
    n_samples=500
)
labelled_training_data = classifier.process_all_postings(
    checkpoint_file="../data/gpt_classified_job_descriptions.csv"
)

Loaded checkpoint with 160 processed postings
Processing 340 postings sequentially...


Processing postings:  15%|█▍        | 50/340 [16:50<1:39:56, 20.68s/it]


Checkpoint saved at 50 postings


Processing postings:  29%|██▉       | 100/340 [33:20<1:25:54, 21.48s/it]


Checkpoint saved at 100 postings


...


Checkpoint saved at 250 postings


Processing postings:  88%|████████▊ | 300/340 [1:56:17<13:52, 20.81s/it]


Checkpoint saved at 300 postings


Processing postings: 100%|██████████| 340/340 [3:55:39<00:00, 41.59s/it]    



Processing complete!
Successfully processed: 340
Failed: 0
Total sentences categorized: 21623
